<a href="https://colab.research.google.com/github/cyberinnovator/NLP_project-/blob/main/Summarization_keyword_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install keybert

In [7]:
!pip install newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.5 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=30348050164569322bd5ee04ea3f0186d77c2d93379e4797c435240919559a6d
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=8d6de5b8f81b27ed2df669b4b9fe9c65c7011dc357255de3e42ab9dc2a594e27
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc933

In [3]:
!pip install lxml[html_clean]

In [8]:
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from keybert import KeyBERT
from newspaper import Article

# Load summarization model and tokenizer

In [11]:
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# Load KeyBERT model
kw_model = KeyBERT(model='all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Content Fetching

In [ ]:
#def fetch_article_content(url):
#    response = requests.get(url)
#    soup = BeautifulSoup(response.content, 'html.parser')
#    # Extract the main content (this might need tuning for different websites)
#    paragraphs = soup.find_all('p')
#    content = " ".join([para.get_text() for para in paragraphs])
#    return content

In [1]:
def fetch_article_content(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

#summarize_and_extract_keywords

In [12]:
def summarize_and_extract_keywords(url):
    try:
        # Fetch article content
        content = fetch_article_content(url)

        # Summarize article
        inputs = tokenizer(content, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Extract keywords using KeyBERT
        keywords = kw_model.extract_keywords(content, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=10)
        keywords = [kw[0] for kw in keywords]  # Extract keyword phrases only

        return summary, keywords
    except Exception as e:
        return f"Error: {str(e)}", []

#try some from here
https://edition.cnn.com/2024/12/27/europe/russia-ukraine-war-drones-alabuga-factory-intl-invs/index.html

https://www.bbc.com/news/science-environment-56837908

https://www.bbc.com/news/articles/cg525d9jlvlo

https://www.bbc.com/news/articles/cwypyg4vq8ko



#main func

#for 1st link

In [13]:
# Input the URL of the article
article_url = input("Enter the URL of the article ")

# Process the URL
summary, keywords = summarize_and_extract_keywords(article_url)

# Display results
print("\nSummary:")
print(summary)
print("\nKeywords:")
print(", ".join(keywords))


Enter the URL of the article https://www.bbc.com/news/articles/cwypyg4vq8ko

Summary:
Million year-old bubbles could solve ice age mystery. Suspended inside the ice are ancient air bubbles which scientists hope will help solve an enduring mystery about our planet's climate history.

Keywords:
oldest ice, ice age, ancient ice, ice ancient, ago glacial, antarctic survey, caves ice, glacial cycles, frozen continent, glacial
